In [17]:
from agents import Agent, trace, Runner, gen_trace_id, function_tool
from agents.model_settings import ModelSettings
from pydantic import BaseModel, Field
from dotenv import load_dotenv
import asyncio
import sendgrid
import os
from sendgrid.helpers.mail import Mail, Email, To, Content
from typing import Dict
from IPython.display import Markdown ,display

import requests

### Defin an search agents


In [3]:
INSTRUCTIONS = "You are a research assistant. Given a search term, you search the web for that term and \
produce a concise summary of the results. The summary must 2-3 paragraphs and less than 300 \
words. Capture the main points. Write succintly, no need to have complete sentences or good \
grammar. This will be consumed by someone synthesizing a report, so it's vital you capture the \
essence and ignore any fluff. Do not include any additional commentary other than the summary itself."


In [5]:
from langchain.tools import DuckDuckGoSearchRun

In [27]:
@function_tool
def duckduckgo_search(query: str) -> str:
    """Search the web using DuckDuckGo (free API)."""
    resp = requests.get("https://api.duckduckgo.com/",
                        params={"q": query, "format": "json"}).json()
    
    results = [r.get("Text") for r in resp.get("RelatedTopics", []) if r.get("Text")]
    print(results)
    return "\n".join(results[:5]) if results else "No results found."


In [28]:
search = DuckDuckGoSearchRun()

In [29]:
search_agent = Agent(
    model = 'gpt-4o-mini',
    instructions = INSTRUCTIONS,
    tools = [duckduckgo_search],
    model_settings = ModelSettings(tool_choice ='required'),
    name = 'search engine'
)

In [30]:
message = "Latest AI Agent frameworks in 2025"


with trace("search"):
    result = await Runner.run(search_agent, message)

[]
[]


In [26]:
display(Markdown(result.final_output))

No current results available on the latest AI Agent frameworks for 2025. Consider checking back later or using different keywords to explore the topic further.

In [31]:
HOW_MANY_SEARCHES = 3

INSTRUCTIONS = f"You are a helpful research assistant. Given a query, come up with a set of web searches \
to perform to best answer the query. Output {HOW_MANY_SEARCHES} terms to query for."


In [32]:
INSTRUCTIONS

'You are a helpful research assistant. Given a query, come up with a set of web searches to perform to best answer the query. Output 3 terms to query for.'

In [33]:
class WebSearchItem(BaseModel):
    reason : str = Field(description="Your reasoning for why this search is important to the query.")

    query :str = Field(description="the search term to use for the websearch")

In [58]:
class WebSearchPlan(BaseModel):
    searches : list[str] = Field(description="A list of web searches to perform to best answer the query.")


planner_Agent = Agent(
    name = "Planner Agent",
    instructions = INSTRUCTIONS,
    model = 'gpt-4o-mini',
    output_type = WebSearchPlan
)

In [59]:
result = await Runner.run(planner_Agent, "Latest AI Agent framworks in 2025")

In [60]:
result.final_output

WebSearchPlan(searches=['latest AI agent frameworks 2025', 'AI agent developments', 'top AI technologies 2025', 'AI frameworks in industry', 'future of AI agents 2025'])

In [61]:
@function_tool
def send_email(subject: str, html_body: str) -> Dict[str, str]:
    """ Send out an email with the given subject and HTML body """
    sg = sendgrid.SendGridAPIClient(api_key=os.environ.get('SENDGRID_API_KEY'))
    from_email = Email("ed@edwarddonner.com") # Change this to your verified email
    to_email = To("ed.donner@gmail.com") # Change this to your email
    content = Content("text/html", html_body)
    mail = Mail(from_email, to_email, subject, content).get()
    response = sg.client.mail.send.post(request_body=mail)
    return {"status": "success"}

In [62]:
INSTRUCTIONS = """You are able to send a nicely formatted HTML email based on a detailed report.
You will be provided with a detailed report. You should use your tool to send one email, providing the 
report converted into clean, well presented HTML with an appropriate subject line."""

email_agent = Agent(
    name="Email agent",
    instructions=INSTRUCTIONS,
    tools=[send_email],
    model="gpt-4o-mini",
)



In [63]:
INSTRUCTIONS = (
    "You are a senior researcher tasked with writing a cohesive report for a research query. "
    "You will be provided with the original query, and some initial research done by a research assistant.\n"
    "You should first come up with an outline for the report that describes the structure and "
    "flow of the report. Then, generate the report and return that as your final output.\n"
    "The final output should be in markdown format, and it should be lengthy and detailed. Aim "
    "for 5-10 pages of content, at least 1000 words."
)


class ReportData(BaseModel):
    short_summary: str = Field(description="A short 2-3 sentence summary of the findings.")

    markdown_report: str = Field(description="The final report")

    follow_up_questions: list[str] = Field(description="Suggested topics to research further")


writer_agent = Agent(
    name="WriterAgent",
    instructions=INSTRUCTIONS,
    model="gpt-4o-mini",
    output_type=ReportData,
)

In [ ]:
async def plan_searches(query:str):
     """ Use the planner_agent to plan which searches to run for the query """
     print("Planning searches...")
     result = await Runner.run(planner_Agent, f"Query :{query}")
     print(f"Will perform {len(result.final_output.searches)} searches")

     return result.final_output


async def perform_searches(search_plan: WebSearchPlan):
     """ Call search() for each item in the search plan """
     print("Searching...")
     print(search_plan.searches)
     tasks = [asyncio.create_task(search(item)) for item in search_plan.searches]
     
     results = await asyncio.gather(*tasks)

     return results


async def search(item: WebSearchItem):
     """ Use the search agent to run a web search for each item in the search plan """
     input = f"Search term: {item} \nReason for searching: {item.reason}"
     result = await Runner.run(search_agent, input)
     return result.final_output





In [72]:
async def write_report(query: str, search_results: list[str]):
    """ Use the writer agent to write a report based on the search results"""
    print("Thinking about report...")
    input = f"Original query: {query}\nSummarized search results: {search_results}"
    result = await Runner.run(writer_agent, input)
    print("Finished writing report")
    return result.final_output

async def send_email(report: ReportData):
    """ Use the email agent to send an email with the report """
    print("Writing email...")
    result = await Runner.run(email_agent, report.markdown_report)
    print("Email sent")
    return report

In [73]:
query ="Latest AI Agent frameworks in 2025"

with trace("Research trace"):
    print("Starting research...")
    search_plan = await plan_searches(query)
    search_results = await perform_searches(search_plan)
    report = await write_report(query, search_results)
    await send_email(report)  
    print("Hooray!")

Starting research...
Planning searches...
Will perform 10 searches
Searching...
['latest AI agent frameworks 2025', 'AI agent advancements 2025', 'top AI development platforms 2025', 'AI agents features and capabilities 2025', 'case studies of AI agents 2025', 'comparison of AI frameworks 2025', 'industry applications of AI agents 2025', 'AI agents software architecture 2025', 'future trends in AI agents', 'AI agent challenges and solutions 2025']


AttributeError: 'str' object has no attribute 'query'